# Problem 2: MNIST: Hand-written digit classification

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

## Loading MNIST
The `torchvision` module contains links to many standard datasets. We can load the MNIST dataset into a `Dataset` object as follows:

In [ ]:
train_dataset = datasets.MNIST('./data', train=True, download=True,  # Downloads into a directory ../data
                               transform=transforms.ToTensor())
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, 
                                                             [int(len(train_dataset)*0.8), int(len(train_dataset)*0.2)], 
                                                             generator=torch.Generator().manual_seed(42))
test_dataset = datasets.MNIST('./data', train=False, download=False,  # No need to download again
                              transform=transforms.ToTensor())

The `Dataset` object is an iterable where each element is a tuple of (input `Tensor`, target):

In [ ]:
type(train_dataset[0][0])

In [ ]:
train_dataset[0][0].size()

In [ ]:
len(train_dataset), len(valid_dataset), len(test_dataset)

In [ ]:
len(torch.utils.data.DataLoader(train_dataset).dataset)

We can convert images to numpy arrays and plot them with matplotlib:

In [ ]:
plt.imshow(train_dataset[0][0][0].numpy(), cmap='gray')

## Example linear model
Let's instantiate a linear model and take a look at the layers.

In [ ]:
model = nn.Sequential(
    # For fully-connected networks, we don't use the 2D structure of an image at all. Our network
    # takes in a flat vector of the pixel values as input.
    nn.Flatten(),  
    nn.Linear(784, 10),
)
print(model)

## Training and validation

Now let's try to train the linear model as an example.

In [ ]:
def run_training_loop(model, batch_size=32, n_epochs=10, lr=1e-3):

    # We could write our training procedure manually and directly index the `Dataset` objects,
    # but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    
    device = "mps" if torch.backends.mps.is_available() else "cpu"

    # Choose Adam as the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Use the cross entropy loss function
    loss_fn = nn.CrossEntropyLoss()

    # store metrics
    train_loss_history = np.zeros([n_epochs, 1])
    valid_accuracy_history = np.zeros([n_epochs, 1])
    valid_loss_history = np.zeros([n_epochs, 1])

    for epoch in range(n_epochs):

        # Some layers, such as Dropout, behave differently during training
        model.train()

        train_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data, target = data.to(device), target.to(device)

            # Erase accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = loss_fn(output, target)
            train_loss += loss.item()

            # Backward pass
            loss.backward()
            
            # Weight update
            optimizer.step()

        train_loss_history[epoch] = train_loss / len(train_loader.dataset)

        # Track loss each epoch
        print('Train Epoch: %d  Average loss: %.4f' %
              (epoch + 1,  train_loss_history[epoch]))

        # Putting layers like Dropout into evaluation mode
        model.eval()

        valid_loss = 0
        correct = 0

        # Turning off automatic differentiation
        with torch.no_grad():
            for data, target in valid_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                valid_loss += loss_fn(output, target).item()  # Sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
                correct += pred.eq(target.view_as(pred)).sum().item()

        valid_loss_history[epoch] = valid_loss / len(valid_loader.dataset)
        valid_accuracy_history[epoch] = correct / len(valid_loader.dataset)

        print('Valid set: Average loss: %.4f, Accuracy: %d/%d (%.4f)\n' %
              (valid_loss_history[epoch], correct, len(valid_loader.dataset),
              100. * valid_accuracy_history[epoch]))
    
    return model, train_loss_history, valid_loss_history, valid_accuracy_history

In [ ]:
model.to(device)
trained_model, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model)
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()
plt.show()

## Testing
After selecting hyperparameters from the validation set, evaluate on the held-out test set (distinct from the validation set).

In [ ]:
def test_performance(model, batch_size=32):
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True) 

    # Putting layers like Dropout into evaluation mode
    model.eval()
    # Use the cross entropy loss function
    loss_fn = nn.CrossEntropyLoss()

    test_loss = 0
    correct = 0

    # Turning off automatic differentiation
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)

    print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
          (test_loss, correct, len(test_loader.dataset),
          100. * test_accuracy))
    return test_loss, test_accuracy

In [ ]:
model.to(device)
test_performance(model)

Not bad! Even the linear model gets above 92% accuracy.

## Visualizing predictions

We can visualize and inspect the model predictions. Here is an example of how to visualize the wrongly classified cases among the first 20 validation examples.

In [ ]:
# Putting layers like Dropout into evaluation mode
model.eval()

# Turning off automatic differentiation
with torch.no_grad():
      for i in range(20):
        data, target = valid_dataset[i]
        model.to("cpu")
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        if pred.item() != target:
            plt.imshow(data[0].numpy(), cmap='gray')
            plt.title(f'Predicted: {pred.item()}')
            plt.show()

# Your code: 2 (b)

In [ ]:
model = nn.Sequential(
    # In problem 2, we don't use the 2D structure of an image at all. Our network
    # takes in a flat vector of the pixel values as input.
    nn.Flatten(),  
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 62),
    nn.ReLU(),
    nn.Linear(62, 10),
)

In [ ]:
trained_model, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model, n_epochs=10)
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()
plt.show()

test_performance(model)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(train_loss_history, label='train')
ax.plot(valid_loss_history, label='valid')
ax.set_xlabel("Epoch Number")
ax.set_ylabel("Cross-Entropy Loss")
plt.title("Training and Validation Loss Curves")
plt.legend()
plt.savefig("loss_curves_q2b.jpg", dpi=300)

# Example code for convolutional neural networks

In [ ]:
# sample model
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 8, kernel_size=(8,8)),
    nn.ReLU(),
    nn.MaxPool2d(8),
    nn.Dropout(p=0.5),
    
    nn.Flatten(),
    nn.Linear(32, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
    # PyTorch implementation of cross-entropy loss includes softmax layer
)

# take a look at the shape of the weights for each layer
for p in model.parameters():
    print(p.data.shape)


# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

In [ ]:
trained_model, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model)
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()
plt.show()

test_performance(model)

## Your code: 2 (c)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(1, 6, 5), # (1, 28, 28) -> (6, 24, 24)
    nn.BatchNorm2d(num_features=6),
    nn.ReLU(),
    nn.MaxPool2d(2, 2), # (6, 24, 24) -> (6, 12, 12)
    nn.Conv2d(6, 16, 5), # (6, 12, 12) -> (16, 8, 8)
    nn.ReLU(),
    nn.MaxPool2d(2, 2), # (16, 8, 8) -> (16, 4, 4)
    nn.Flatten(),
    nn.Linear(16 * 4 * 4, 120),
    nn.ReLU(),
    nn.Linear(120, 84),
    nn.Dropout(p=0.1),
    nn.ReLU(),
    nn.Linear(84, 10),
)

# make sure the # of parameters is under 50,000
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

In [ ]:
trained_model, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model)
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()
plt.show()

test_performance(model)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(train_loss_history, label='train')
ax.plot(valid_loss_history, label='valid')
ax.set_xlabel("Epoch Number")
ax.set_ylabel("Cross-Entropy Loss")
plt.title("Training and Validation Loss Curves")
plt.legend()
plt.savefig("loss_curves_q2c.jpg", dpi=300)

In [ ]:
plt.imshow(model[0].weight.detach().numpy()[0][0], cmap='gray')

# Problem 3: Vision Transformer

In [ ]:
!pip install einops

import torch
from torch import nn

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

## Your code: 3(a)

Implement single-head attention.

In [ ]:
class SingleHeadAttention(nn.Module):
    def __init__(self, input_dim, inner_dim, dropout = 0.):
        super().__init__()
        self.Q = nn.Linear(input_dim, inner_dim, bias=False)
        self.K = nn.Linear(input_dim, inner_dim, bias=False)
        self.V = nn.Linear(input_dim, inner_dim, bias=False)
        self.softmax = nn.Softmax(dim=1)
        self.scale = 1 / np.sqrt(inner_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        Q_proj = self.Q(x)
        K_proj = self.K(x)
        V_proj = self.V(x)

        K_T = K_proj.transpose(1, 2)
        QK_T = torch.matmul(Q_proj, K_T) * self.scale
        attention = self.softmax(QK_T)
        self.attention_cache = attention
        out = torch.matmul(attention, V_proj)
        out = self.dropout(out)

        return out

Test the following multihead attention implementation that relies on the single-head attention implementation above.

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        self.attention_heads = nn.ModuleList([
            SingleHeadAttention(dim, dim_head, dropout=dropout)
            for _ in range(heads)
        ])

    def forward(self, x):
        out = torch.cat([
            head(x) for head in self.attention_heads
        ], dim=-1)
        return out

Test code for multihead attention:

In [ ]:
T = 5
input_dim = 32
batch_size = 20

heads = 8
dim_head = 64

test_input = torch.zeros((batch_size, T, input_dim))

test_attention_module = Attention(input_dim)
test_output = test_attention_module(test_input)
print(test_input.shape)

print(test_output)
print(test_output.size())
assert test_output.size() == (batch_size, T, heads * dim_head), "Shapes are incorrect"

## Your code: 3 (b)

In [ ]:
class ViTLayer(nn.Module):
    def __init__(self, dim, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.attn = nn.Sequential(
            nn.LayerNorm(dim),
            Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
            nn.Dropout(p=dropout),
            nn.Linear(heads * dim_head, dim, bias=False)
        )
        self.feedforward = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, mlp_dim),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(mlp_dim, dim),
            nn.Dropout(p=dropout),
        )

    def forward(self, x):
        x = x + self.attn(x)
        x = x + self.feedforward(x)

        return x

## ViT code for 3 (c)

In [ ]:
class ViT(nn.Module):
    def __init__(self, patch_size=7, dim=64, depth=6, heads=8, mlp_dim=128, 
                 dim_head = 64,
                 dropout = 0., emb_dropout = 0.):
        super().__init__()

        image_height, image_width = 28, 28
        num_classes = 10
        channels = 1

        patch_height = patch_size
        patch_width = patch_size

        assert image_height % patch_height == 0 and image_width % patch_width == 0, (
            'Image dimensions must be divisible by the patch size.')

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.emb_dropout = nn.Dropout(emb_dropout)

        self.transformer = nn.Sequential(*nn.ModuleList([
            ViTLayer(dim, heads, dim_head, mlp_dim, dropout)
            for _ in range(depth)                       
        ]))

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.emb_dropout(x)

        x = self.transformer(x)

        x = x[:, 0]
        return self.mlp_head(x)

In [ ]:
model = ViT(patch_size=7, dim=64, depth=6, heads=8, mlp_dim=128)
model.to(device)

In [ ]:
trained_model, train_loss_history, valid_loss_history, valid_accuracy_history = run_training_loop(model)
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()
plt.show()

test_performance(model)

In [ ]:
torch.save(model.state_dict(), "ViT.pth")

In [ ]:
X, y = valid_dataset[0]

In [ ]:
plt.imshow(X.detach().numpy().squeeze(), cmap='gray')

In [ ]:
model(X.unsqueeze(0).to(device))

In [ ]:
heads = model.transformer[-1].attn[1].attention_heads

In [ ]:
heads[0].attention_cache.detach().cpu().numpy().squeeze()

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(8,4))

for i, (head, ax) in enumerate(zip(heads, axes.flatten()), 1):
    ax.imshow(head.attention_cache.detach().cpu().numpy().squeeze(), cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel(f"Head {i}")
    
plt.suptitle("Attention Heads (First Valid)");
plt.savefig("attention_heads.jpg", dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(train_loss_history, label='train')
ax.plot(valid_loss_history, label='valid')
ax.set_xlabel("Epoch Number")
ax.set_ylabel("Cross-Entropy Loss")
plt.title("Training and Validation Loss Curves")
plt.legend()
plt.savefig("loss_curves_q3c.jpg", dpi=300)